In [0]:
import pandas as pd
import numpy as np


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving churn.csv to churn.csv
User uploaded file "churn.csv" with length 323787 bytes


In [0]:
data=pd.read_csv('churn.csv')

In [8]:
data.head()

,Unnamed: 0,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,0,KS,128,415,382-4657,no,yes,25,265.1,110,?,197.4,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False.
1,1,OH,107,415,371-7191,no,yes,26,161.6,123,?,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False.
2,2,NJ,137,415,358-1921,no,no,0,243.4,114,?,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,3,OH,84,408,375-9999,yes,no,0,299.4,71,50.9,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [0]:
data = data.replace('?', np.NaN)

In [10]:
# Count total NA's in each column
print(data.isnull().sum())

Unnamed: 0          0
State               0
Account Length      0
Area Code         173
Phone               0
Int'l Plan        528
VMail Plan        349
VMail Message       0
Day Mins            0
Day Calls           0
Day Charge        112
Eve Mins           88
Eve Calls         520
Eve Charge          0
Night Mins          0
Night Calls         0
Night Charge      310
Intl Mins           0
Intl Calls        662
Intl Charge       352
CustServ Calls      0
Churn?              0
dtype: int64


In [0]:
# delete the column having no importance or have alot of Na's
data1 = data.drop(['Phone', 'VMail Plan', 'State','Unnamed: 0'], axis=1)

In [0]:
# Drop the rows of column initaial plan containing NA's
data2 = data1.dropna(subset=["Int'l Plan"])

# encode text to number

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data2["Int'l Plan"])
data2["Int'l Plan"] = le.transform(data2["Int'l Plan"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data2["Churn?"])
data2["Churn?"] = le.transform(data2["Churn?"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [20]:
data2.head()

,Account Length,Area Code,Int'l Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,128,415,0,25,265.1,110,NaN,197.4,99,16.78,244.7,91,11.01,10.0,3,2.7,1,0
1,107,415,0,26,161.6,123,NaN,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,0
2,137,415,0,0,243.4,114,NaN,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,408,1,0,299.4,71,50.9,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,415,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [21]:
data2.columns

Index(['Account Length', 'Area Code', 'Int'l Plan', 'VMail Message',
       'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls',
       'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins',
       'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?'],
      dtype='object')

In [0]:
col=['Account Length', 'Area Code', "Int'l Plan", 'VMail Message',
       'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls',
       'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins',
       'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?']

# replace nan with mean()

In [24]:
from sklearn.preprocessing import Imputer

mean_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

# Train the imputor on the df dataset
mean_imputer = mean_imputer.fit(data2)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
imputed_df = mean_imputer.transform(data2.values)

In [0]:
df = pd.DataFrame(imputed_df)

In [0]:
df.columns = col

In [48]:
df.head()

,Account Length,Area Code,Int'l Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,128.0,415.0,0.0,25.0,265.1,110.0,30.690229,197.4,99.0,16.78,244.7,91.0,11.01,10.0,3.0,2.70,1.0,0
1,107.0,415.0,0.0,26.0,161.6,123.0,30.690229,195.5,103.0,16.62,254.4,103.0,11.45,13.7,3.0,3.70,1.0,0
2,137.0,415.0,0.0,0.0,243.4,114.0,30.690229,121.2,110.0,10.30,162.6,104.0,7.32,12.2,5.0,3.29,0.0,0
3,84.0,408.0,1.0,0.0,299.4,71.0,50.900000,61.9,88.0,5.26,196.9,89.0,8.86,6.6,7.0,1.78,2.0,0
4,75.0,415.0,1.0,0.0,166.7,113.0,28.340000,148.3,122.0,12.61,186.9,121.0,8.41,10.1,3.0,2.73,3.0,0


In [0]:
# convert the target variable from float to int type
df["Churn?"] = df["Churn?"].astype(int)

In [50]:
# Check if all the null has been imputed
print(df.isnull().sum())

Account Length    0
Area Code         0
Int'l Plan        0
VMail Message     0
Day Mins          0
Day Calls         0
Day Charge        0
Eve Mins          0
Eve Calls         0
Eve Charge        0
Night Mins        0
Night Calls       0
Night Charge      0
Intl Mins         0
Intl Calls        0
Intl Charge       0
CustServ Calls    0
Churn?            0
dtype: int64


In [0]:
# Splitting the data into train and test
from sklearn.model_selection import train_test_split

In [0]:
index = ["Account Length", "Area Code", "Int'l Plan","VMail Message" ,"Day Mins", "Day Calls", 
                "Day Charge", "Eve Mins", "Eve Calls", "Eve Charge", "Night Mins", "Night Calls", "Night Charge",
                 "Intl Mins", "Intl Calls", "Intl Charge", "CustServ Calls", "Churn?"]
X = df[index]

y = df["Churn?"]

train, test = train_test_split(X, test_size=0.3, random_state=4)

In [0]:
index1 = ["Account Length", "Area Code", "Int'l Plan","VMail Message" ,"Day Mins", "Day Calls", 
                "Day Charge", "Eve Mins", "Eve Calls", "Eve Charge", "Night Mins", "Night Calls", "Night Charge",
                 "Intl Mins", "Intl Calls", "Intl Charge", "CustServ Calls"]

In [0]:
x_train = train[index1]
y_train = train['Churn?']
x_test = test[index1]
y_test = test['Churn?']



In [139]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
seed = 1
clf = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth= 6, min_samples_split=3, min_samples_leaf=10, 
                             max_features= 'auto', max_leaf_nodes= None, min_impurity_decrease=0.0,min_impurity_split=None, 
                             n_jobs=1, class_weight=None)
model = clf.fit(x_train, y_train)

result = model.predict(x_test)
print('Accuracy=',np.mean(result == y_test))

Accuracy= 0.9109263657957245


In [140]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test, result, pos_label=1)

metrics.auc(fpr, tpr)

0.7031935889001854

#   diffrent approach

In [141]:
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

clf = RandomForestClassifier()

param_grid = {
                 'n_estimators': [5, 10, 15, 20],
                 'max_depth': [2, 5, 7, 9]
             }


grid_clf = GridSearchCV(clf, param_grid ,cv=10)
grid_clf.fit(x_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [142]:
y_pred=grid_clf.predict(x_test)

print('Accuracy=',np.mean(y_pred == y_test))


Accuracy= 0.9477434679334917


In [144]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)

print('AUC=',metrics.auc(fpr, tpr))

AUC= 0.8416362657735782
